- 대표 기사 추출법의 전제
  1. 어느 중요한 사건에 대해
  2. 짧은 기간 안에 같은 주제에 대해 여러 기사들이 발행되었다면 중요한 주제일 가능성이 높다.
  - 임의의 기사들이 같은 주제에 대해 다루고 있는지 판별하기  

    
- cosine similarity의 의미
  - cosine similarity가 높으면 높을 수록 기사 내 비슷한 중요 단어들이 더 많다는 뜻이다. 의미없는 불용어(stopword)들은 제거를 한 후이기 때문에 비슷한 단어들이 많을수록 기사가 유사할 가능성이 높아진다.
  - 연간 대표 기사 갯수를 ~=10개로 조정하기 위해 cosine similarity를 조절(0.3 이상)하였다. 더 많은 기사들 추출을 원한다면 similarity를 낮추면 되고, 더 적은 기사들을 원한다면 similarity를 높이면 된다. 하지만 너무 낮춘다면 자칫 관계없는 기사들끼리 엮일 수가 있다.

In [ ]:
# libraries to install
# !pip install rhinoMorph konlpy

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import rhinoMorph
from konlpy.tag import Okt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### Raw article corpus preprocessing
### 로우 기사 데이터 전처리

In [3]:
DATA_DIR = '/root/../content/drive/MyDrive/bts_project/data'

In [4]:
# calling news corpora data crawled from Naver News
columns = ['media','date','title','article_original','url']
news_df = pd.read_excel(DATA_DIR + '/news_df_210222_v05.xlsx')
news_df.head()

,media,date,title,article_original,url
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,2012년 싸이에 이어 두 번째 ABC방송 라이브 쇼에서 생중계 전 지구를 홀린 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,방탄소년단BTS이 미국 CNN 선정 2010년대 음악을 변화시킨 10대 아티스트에...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,트와이스 도쿄돔 공연 블랙핑크도 새앨범 예정 경자년 새해에도 K팝 스타들의 세계...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 K팝 명소들을 선정해...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [5]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   media             861 non-null    object
 1   date              861 non-null    object
 2   title             861 non-null    object
 3   article_original  853 non-null    object
 4   url               861 non-null    object
dtypes: object(5)
memory usage: 33.8+ KB


In [6]:
# checking null items
news_df.isna().sum()

media               0
date                0
title               0
article_original    8
url                 0
dtype: int64

In [7]:
# deleting null items and resetting index
news_df.dropna(inplace=True)
news_df.reset_index(drop=True, inplace=True)
news_df.info()
news_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853 entries, 0 to 852
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   media             853 non-null    object
 1   date              853 non-null    object
 2   title             853 non-null    object
 3   article_original  853 non-null    object
 4   url               853 non-null    object
dtypes: object(5)
memory usage: 33.4+ KB


,media,date,title,article_original,url
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,2012년 싸이에 이어 두 번째 ABC방송 라이브 쇼에서 생중계 전 지구를 홀린 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,방탄소년단BTS이 미국 CNN 선정 2010년대 음악을 변화시킨 10대 아티스트에...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,트와이스 도쿄돔 공연 블랙핑크도 새앨범 예정 경자년 새해에도 K팝 스타들의 세계...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 K팝 명소들을 선정해...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한...,https://news.naver.com/main/read.nhn?mode=LSD&...


### Calculating TF-IDF  
### TF-IDF 계산

In [8]:
# importing stopwords
# reference: https://www.ranks.nl/stopwords/korean

stopwords = pd.read_csv(DATA_DIR + '/stopwords.txt', header=None)
print(stopwords)

       0
0      는
1      아
2      휴
3    아이구
4    아이쿠
..   ...
672   여덟
673   아홉
674    령
675    영
676    은

[677 rows x 1 columns]


In [9]:
### noun extraction using Okt & stopwords removal
### Okt를 이용한 기사별 중요 명사 추출 & 불용어 제거

def okt_morph(articles):
  '''
  Extracting important nouns per article.
  Returns morphed version of articles.
  '''
  okt = Okt()
  articles_morphed = []
  for article in articles:
      morph = okt.nouns(article)

      # stopwords elimination & combines nouns into a single string
      # 불용어 제거 & 명사들을 단일 스트링으로 변환
      stopped = ''
      for word in morph:
          if word not in stopwords:
              stopped += word
              stopped += ' '

      articles_morphed.append(stopped)
  return articles_morphed

news_df['article_morphed'] = okt_morph(news_df.article_original)

In [10]:
# example of morphed articles
# 변환된 기사 예시
news_df.article_morphed[0:20]

0     싸이 두 방송 라이브 쇼 생중계 전 지구 홀린 그룹 사회자 소개 방탄소년단 이 모습...
1     방탄소년단 이 미국 선정 음악 변화 아티스트 선정 은 자신 음악 장르 스스로 재창 ...
2     트와이스 도쿄돔 공연 블랙 핑크 앨범 예정 경자년 새해 팝 스타 세계 무대 공략 계...
3     팝 대한 관심 전 세계 서울시 팝 명소 선정 홍보 멀리 한국 외국인 관광객 방탄소년...
4     골든글로브 시상식 한국 최초 외국어 영화상 수상한 기생충 봉준호 감독 매체 인터뷰 ...
5     차세대 유니콘 스마트 스터디 김민석 대표 삼성 출판사 세로 더 유명 유아 콘텐츠 게...
6     상표권 그룹 방탄소년단 소속사 빅히트 엔터테인먼트 이하 빅히트 분쟁 벌 신세계 상표...
7     방탄소년단 소속사 빅히트 엔터테인먼트 비 티에스 상표권 분쟁 중 신세계 백화점 관련...
8     그룹 방탄소년단 이 다음 달 새 앨범 소속사 빅히트 엔터테인먼트 오전 팬 커뮤니티 ...
9     그룹 방탄소년단 이 다음 달 새 앨범 소속사 빅히트 엔터테인먼트 오전 팬 커뮤니티 ...
10    그룹 방탄소년단 사진 다음 달 새 앨범 소속사 빅히트 엔터테인먼트 팬 커뮤니티 위 ...
11    방탄소년단 이 새 앨범 맵 오브 더 솔 발매 앞서 선공 곡 발표 방탄소년단 오전 소...
12    방탄소년단 컴백 일정 발표 영향 영어권 네티즌 특정 어휘 평소 대비 더 검색 것 미...
13    다음 달 새 앨범 그룹 방탄소년단 의 컴백 트레일러 섀도 공개 방탄소년단 소속사 빅...
14    방탄소년단 이 처음 유튜브 뮤직비디오 보유 소속사 빅히트 엔터테인먼트 이 러브 유 ...
15    방탄소년단 이 지난해 발매 맵 오브 더 솔 페르소나 역대 최 판매량 가온차트 연간 ...
16    실력 파 출동 미스터 트롯 김준수 도플갱어 크론병 영기 등 심사 위원 전원 합격 명...
17    나 이상 가족 울 보이밴드 팬 얼마 전 입덕 작년 말 서울 팝업 스토어 올해

### Selecting important articles  
### 대표기사 추출

In [11]:
# changing date column into pandas 'date' format
# 판다스 'date' 형식으로 날짜 column 변환
news_df['date_pd'] = pd.to_datetime(news_df['date'])
news_df.drop(columns='date', axis=1,inplace=True)

# grouping articles per month
# 월간별 기사 grouping
grouped_indice = []  # stores grouped articles' indice
date_cuts = pd.date_range(start='2020-01-01', end='2020-12-31', periods=13)
num_of_periods = len(date_cuts) - 1
print("Date cuts:", date_cuts, "\nNumber of groups:", num_of_periods)
print("Number of total articles:", len(news_df))
for i, date in enumerate(date_cuts):
  index_group = []
  if i < len(date_cuts) - 1:
    for j in range(len(news_df)):
      if date_cuts[i] < news_df.date_pd[j] <= date_cuts[i+1]:
        index_group.append(j)
    grouped_indice.append(index_group)

print("Number of articles per monthly period:\n", {i+1 : len(indice) for i, indice in enumerate(grouped_indice)})

Date cuts: DatetimeIndex(['2020-01-01 00:00:00', '2020-01-31 10:00:00',
               '2020-03-01 20:00:00', '2020-04-01 06:00:00',
               '2020-05-01 16:00:00', '2020-06-01 02:00:00',
               '2020-07-01 12:00:00', '2020-07-31 22:00:00',
               '2020-08-31 08:00:00', '2020-09-30 18:00:00',
               '2020-10-31 04:00:00', '2020-11-30 14:00:00',
               '2020-12-31 00:00:00'],
              dtype='datetime64[ns]', freq=None) 
Number of groups: 12
Number of total articles: 853
Number of articles per monthly period:
 {1: 55, 2: 69, 3: 45, 4: 21, 5: 44, 6: 56, 7: 21, 8: 49, 9: 189, 10: 172, 11: 67, 12: 65}


In [12]:
# tfidf transformation per date group
def tfidfy(time_period_no):
  '''
  time_period_no : month number
  '''

  # calling articles corpora using indice
  article_group = []
  for index in grouped_indice[time_period_no]:
    article_group.append(news_df.article_original[index])

  tfidfv = TfidfVectorizer(lowercase=False).fit(article_group)
  transformed = tfidfv.transform(article_group)
  # print(tfidfv.vocabulary_)

  return transformed

tfidfy(0)

<55x6801 sparse matrix of type '<class 'numpy.float64'>'
	with 11314 stored elements in Compressed Sparse Row format>

*checkpoint* -- refactoring

In [14]:
def get_similar_art_index(group_no, art_group_tfidf, idx):
  '''
  Returns the index of important articles.

  group_no : month no
  art_group_tfidf : tfidf transformed articles
  idx : article index
  '''

  sim_pair = cosine_similarity(art_group_tfidf[idx], art_group_tfidf)
  # print(sim_pair.shape)
  sorted_index = sim_pair.argsort()[:,::-1]  # sorting according to indice in descending order
  sorted_index = sorted_index[:,1:]  # eliminating self
  # print(sorted_index)

  art_sim_value = np.sort(sim_pair.reshape(-1))[::-1]  # sorting according to similarity in descending order
  art_sim_value = art_sim_value[1:] # eliminating self
  # print(art_sim_value)

  ref = 0
  for i in range(group_no):
    ref += len(grouped_indice[i])

  index_imp = []
  for idx, value in enumerate(art_sim_value):
    if value >= 0.3:
      group_index = sorted_index[0][idx]
      real_index = ref + group_index
      index_imp.append(real_index)
      sim_counter[real_index] += 1

  return index_imp

In [15]:
#유사 참조 카운터
sim_counter = np.zeros(len(news_df))
#전체 문서에 대해, 날짜 구간 당 유사한 기사의 index를 리스트에 저장하고 카운터를 늘려준다.
sim_list=[]
for group_no in range(len(grouped_indice)):
  for idx in range(len(grouped_indice[group_no])):
    art_group_tfidf = tfidfy(group_no)
    sim_list.append(get_similar_art_index(group_no, art_group_tfidf, idx))

#dataframe에 컬럼 추가
news_df['sim_articles'] = sim_list
news_df['sim_count'] = sim_counter

In [16]:
#유사한 기사가 3개 이상이면 불러오기  --> 없어도 되는 단계
idx_list = []
for i,ls in enumerate(news_df['sim_articles']):
  if len(ls)>=3:
    idx_list.append(i)
#print(idx_list)
news_df.iloc[idx_list]

,media,title,article_original,url,article_morphed,date_pd,sim_articles,sim_count
36,조선일보,방탄소년단 그래미 무대 선다 한국 가수 최초,그룹 방탄소년단BTS이 제62회 그래미 어워즈Grammy Awards에서 공연한다...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 이 제 그래미 어워즈 공연 그래미 어워즈 미국 대중음악 시상식 중 ...,2020-01-24,"[37, 41, 39, 40]",4.0
37,경향신문,방탄소년단 한국 가수 최초 미 그래미 어워드 공연 확정,그룹 방탄소년단BTS이 제62회 그래미 어워드Grammy Awards에서 공연한다...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 이 제 그래미 어워드 공연 최고 권위 그래미 어워드 한국 가수 공연...,2020-01-24,"[36, 41, 39, 40]",4.0
39,한겨레,BTS 한국 가수 최초 그래미 공연내년엔 후보 목표,그룹 방탄소년단이 한국 가수 최초로 미국 그래미 시상식에서 공연했다. 방탄소년단은...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 한국 가수 최초 미국 그래미 시상식 공연 방탄소년단 미국 로스앤젤레...,2020-01-27,"[40, 41, 43, 36, 37]",5.0
40,경향신문,내년엔 그래미 후보될까짧았던 방탄소년단 무대,방탄소년단이 26일현지시간 열린 그래미 시상식 공연 무대에 올랐다. 단독 공연이 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단 그래미 시상식 공연 무대 단독 공연 합동 무대 중 하나 시간 방탄소년단 ...,2020-01-27,"[39, 43, 41, 37, 36]",5.0
41,조선일보,방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전,그룹 방탄소년단BTS이 26일현지 시각 한국 가수 최초로 미국 그래미 시싱식에서 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 이 시각 한국 가수 최초 미국 그래미 싱 공연 방탄소년단 날 미국 ...,2020-01-27,"[39, 36, 37, 40, 43]",5.0
...,...,...,...,...,...,...,...,...
801,중앙일보,한국말 노래로 핫 100 정상 대관식BTS 빌보드 62년 역사 다시 썼다,방탄소년단BTS이 또 한 번 새 역사를 썼다. 우리말로 된 신곡 라이프 고스 온L...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단 이 또 번 새 역사 우리말 신곡 라이프 고스 온 빌보드 싱글 차트 핫 위...,2020-12-02,"[790, 797, 792, 798, 802]",5.0
812,중앙일보,BTS MAMA 8관왕 싹쓸이저희만 좋은 소식 마음 무겁기도,그룹 방탄소년단BTS이 엠넷 아시안 뮤직 어워즈2020 MAMA에서 2년 연속으로...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 이 엠넷 아시안 뮤직 어워즈 연속 대상 개 부문 싹 진 기록 방탄소...,2020-12-07,"[815, 814, 813]",3.0
813,조선일보,BTS MAMA 싹쓸이,그룹 방탄소년단BTS이 아시아 최고 권위의 케이팝 시상식 2020 MAMA엠넷 아...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 이 아시아 최고 권위 케이팝 시상식 엠넷 아시안 뮤직 어워즈 연속 ...,2020-12-07,"[812, 815, 814]",3.0
814,한겨레,BTS 올해도 MAMA서 대상 싹쓸이8개 부문 석권,그룹 방탄소년단BTS이 엠넷 아시안 뮤직 어워즈2020 MAMA에서 2년 연속으로...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 이 엠넷 아시안 뮤직 어워즈 연속 대상 개 부문 차지 이 지난 엠넷...,2020-12-07,"[815, 812, 813]",3.0


In [17]:
#대표 기사 뽑기 (가장 참조가 많이 된 기사)
articles_final = []
for group in grouped_indice:
    indx = group[0]
    for art_indx in group:
        if news_df.sim_count[art_indx] >= news_df.sim_count[indx]:
            indx = art_indx
    articles_final.append(indx)
articles_final

[41, 121, 149, 170, 214, 264, 305, 353, 544, 630, 773, 792]